In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# **Silver_data_transformation**

In [0]:
df = spark.read.format("delta")\
  .option("header", True)\
  .option("inferSchema", True)\
  .load("abfss://bronze@jairamnetflixdatalake.dfs.core.windows.net/netflix_titles")


Handeling nulls with numbers and using different numbers for different columns

In [0]:
df = df.fillna({"duration_minutes":0, "duration_seasons":1})

We willuse with column function to add new column or to modify the existing column

In [0]:
df = df.withColumn("duration_minutes", col("duration_minutes").cast(IntegerType()))
df = df.withColumn("duration_seasons", col("duration_seasons").cast(IntegerType()))


In [0]:
df.printSchema()

root
 |-- duration_minutes: integer (nullable = true)
 |-- duration_seasons: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- description: string (nullable = true)
 |-- show_id: string (nullable = true)
 |-- _rescued_data: string (nullable = true)
 |-- short_title: string (nullable = true)
 |-- type_flag: integer (nullable = false)
 |-- Duration_Raning: integer (nullable = false)



Splliting the movie name with :

In [0]:
df = df.withColumn("short_title", split(col("title"), ":")[0])



/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
df =df.withColumn("rating", split(col("rating"), "-")[0])


/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


### Conditional column

In [0]:
df = df.withColumn("type_flag", when(col("type") == "Movie", 1)\
                                .when(col("type") == "TV Show", 2)\
                                .otherwise(0))


/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


### We are using window function to assign rank.****

In [0]:
from pyspark.sql.window import Window


In [0]:
df =df.withColumn("Duration_Raning", dense_rank().over(Window.orderBy(col("duration_minutes").desc())))



/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


Using of SQlin spark

--> todo that first we have to create a tempview of our data frame

--> if we want to use these views else where we should create global temp views

In [0]:
df.createOrReplaceTempView("Temp_View")

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
df = spark.sql("SELECT * FROM Temp_View")



In [0]:
df.createOrReplaceGlobalTempView("Global_View")
df = spark.sql("SELECT * FROM global_temp.Global_View")


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-7213833162516442>, line 1
----> 1 df.createOrReplaceGlobalTempView("Global_View")
      2 df = spark.sql("SELECT * FROM global_temp.Global_View")

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/dataframe.py:2157, in DataFrame.createOrReplaceGlobalTempView(self, name)
   2153 def createOrReplaceGlobalTempView(self, name: str) -> None:
   2154     command = plan.CreateView(
   2155         child=self._plan, name=name, is_global=True, replace=True
   2156     ).command(session=self._session.client)
-> 2157     _, _, ei = self._session.client.execute_command(command, self._plan.observations)
   2158     self._execution_info = ei

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py:1297, in SparkConnectClient.execute_command(self, command, observations, extra_request

Aggregation funtion to get no of movies and tv shows


In [0]:
df_Vis = df.groupBy("type").agg(count("*").alias("total_count"))
df_Vis.display()

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
df.write.format("delta")\
    .mode("overwrite")\
    .option("path","abfss://silver@jairamnetflixdatalake.dfs.core.windows.net/netflix_titles")\
    .save()

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:434)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:466)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:757)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:510)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:616)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:643)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:49)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:293)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(Attr